# Pipestat CLI

**Before following this tutorial please make sure you're familiar with more information-rich "Pipestat Python API" tutorial.**

## Prepare environment

Pipestat command line interface can use multiple environment variables to avoid copious repetition of arguments in subsequent `pipestat` calls. 

Please refer to the [Environment variables reference](http://pipestat.databio.org/en/latest/env_vars/) for the complete list of supported environment variables. We will set a few for this tutorial:

In [1]:
export PIPESTAT_RESULTS_SCHEMA=../tests/data/sample_output_schema.yaml
export PIPESTAT_RECORD_ID=sample1
export PIPESTAT_RESULTS_FILE=`mktemp` # temporary file for results storage
export PIPESTAT_NAMESPACE=test

## Usage demonstration

### Reporting

Naturally, the command line interface provides access to all the Python API functionalities of `pipestat`. So, for example, to report a result and back the object by a file use:

In [10]:
pipestat report -i number_of_things -v 100 --try-convert

Reported records for 'sample1' in 'test' namespace:
 - number_of_things: 100


The result has been reported and the database file has been updated:

In [11]:
cat $PIPESTAT_RESULTS_FILE

test:
  sample1:
    number_of_things: 100


Let's report another result:

In [12]:
pipestat report -i percentage_of_things -v 1.1 --try-convert

Reported records for 'sample1' in 'test' namespace:
 - percentage_of_things: 1.1


In [13]:
cat $PIPESTAT_RESULTS_FILE

test:
  sample1:
    number_of_things: 100
    percentage_of_things: 1.1


### Inspection

`pipestat inspect` command is a way to briefly look at the general `PipestatManager` state, like number of records, type of backend etc.

In [14]:
pipestat inspect



PipestatManager (test)
Backend: file (/var/folders/3f/0wj7rs2144l9zsgxd3jn5nxc0000gn/T/tmp.Zid7BMd1)
Results schema source: ../tests/data/sample_output_schema.yaml
Status schema source: /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pipestat/schemas/status_schema.yaml
Records count: 1


In order to display the contents of the results file or database table associated with the indicated namespace, add `--data` flag:

In [15]:
pipestat inspect --data



PipestatManager (test)
Backend: file (/var/folders/3f/0wj7rs2144l9zsgxd3jn5nxc0000gn/T/tmp.Zid7BMd1)
Results schema source: ../tests/data/sample_output_schema.yaml
Status schema source: /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pipestat/schemas/status_schema.yaml
Records count: 1

Data:
test:
  sample1:
    number_of_things: 100
    percentage_of_things: 1.1


### Retrieval

Naturally, the reported results can be retrieved. Just call `pipestat retrieve` to do so:

In [16]:
pipestat retrieve -i percentage_of_things

1.1


### Removal

In order to remove a result call `pipestat remove`:

In [17]:
pipestat remove -i percentage_of_things

Removed result 'percentage_of_things' for record 'sample1' from 'test' namespace


The results file and the state of the `PipestatManager` object reflect the removal:

In [18]:
cat $PIPESTAT_RESULTS_FILE

test:
  sample1:
    number_of_things: 100


In [19]:
pipestat inspect --data



PipestatManager (test)
Backend: file (/var/folders/3f/0wj7rs2144l9zsgxd3jn5nxc0000gn/T/tmp.Zid7BMd1)
Results schema source: ../tests/data/sample_output_schema.yaml
Status schema source: /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pipestat/schemas/status_schema.yaml
Records count: 1

Data:
test:
  sample1:
    number_of_things: 100


## Status management

To manage pipeline status call `pipestat status <subcommand>`:

- `set` to set pipeline statuses
- `get` to retrieve pipeline statuses

Starting with `pipestat 0.0.3` the `--schema` argument is not required for status management if YAML file is used as the backend.

In [20]:
pipestat status set --status-identifier running
pipestat status get

running


Note that only statuses defined in the status schema are supported:

In [21]:
rm $PIPESTAT_RESULTS_FILE